In [1]:
#import packages, cd into current working directory
print('Loading libraries...')
print('Loading input...')

import openpyxl
import os
import pandas as pd
import datetime
from datetime import date
import scipy
from scipy import optimize
from scipy.optimize import newton
from decimal import *

os.chdir('Limit3dP4rtnerR3turns') #comment out to work with code chunk and add libraries
print(os.listdir()) #check contents of working directory

current_date = date(2019,6,30)

print('Done')
print("Begin update of LP returns as of " + str(current_date))


Loading libraries...
Loading input...
['Red & Blue PH Annex Portfolio Returns 6 30 19.xlsm', 'Red & Blue I Portfolio Returns 6 30 19.xlsm', '.DS_Store', 'MentorTech 2005 Portfolio Returns 6 30 19.xlsm', '6 30 19 LP Returns Quidsi Annex.csv', '6 30 19 LP Returns MTII.csv', '6 30 19 LP Returns.csv', 'Limit3dP4rtnerR3turns.ipynb', 'LP Accounts.csv', '6 30 19 LP Returns PH Annex.csv', '6 30 19 LP Returns.xlsx', 'Red & Blue SS Annex Portfolio Returns 6 30 19.xlsm', '6 30 19 LP Returns SS Annex.csv', '6 30 19 LP Returns MT2005.csv', '6 30 19 LP Returns MTIII.csv', 'MentorTech Jet Annex Portfolio Returns 6 30 19.xlsm', 'MentorTech Annex Portfolio Returns FINAL.xlsm', '6 30 19 LP Returns R&B1.csv', '.ipynb_checkpoints', 'MentorTech III Portfolio Returns 6 30 19.xlsm', 'MentorTech II Portfolio Returns 6 30 19.xlsm', '6 30 19 LP Returns Jet Annex.csv', '~$6 30 19 LP Returns.xlsx']
Done
Begin update of LP returns as of 2019-06-30


In [2]:
#Load all .xlsm files from local directory into python, rename and comment out as necessary

print('Loading returns sheets...')

excel_MentorTech2005PortfolioReturns = openpyxl.load_workbook('MentorTech 2005 Portfolio Returns 6 30 19.xlsm', data_only=True)
excel_MentorTechIIPortfolioReturns = openpyxl.load_workbook('MentorTech II Portfolio Returns 6 30 19.xlsm', data_only=True)
excel_MentorTechIIIPortfolioReturns = openpyxl.load_workbook('MentorTech III Portfolio Returns 6 30 19.xlsm', data_only=True)
excel_MentorTechAnnexPortfolioReturns = openpyxl.load_workbook('MentorTech Annex Portfolio Returns FINAL.xlsm', data_only=True)
excel_MentorTechJetAnnexPortfolioReturns = openpyxl.load_workbook('MentorTech Jet Annex Portfolio Returns 6 30 19.xlsm', data_only=True)
excel_RedandBlueIPortfolioReturns = openpyxl.load_workbook('Red & Blue I Portfolio Returns 6 30 19.xlsm', data_only=True)
excel_RedandBlueSSAnnexPortfolioReturns = openpyxl.load_workbook('Red & Blue SS Annex Portfolio Returns 6 30 19.xlsm', data_only=True)
excel_RedandBluePHAnnexPortfolioReturns = openpyxl.load_workbook('Red & Blue PH Annex Portfolio Returns 6 30 19.xlsm', data_only=True)

print('Done')

Loading returns sheets...
Done


In [3]:
#Parse cash flow info into a data frame: data = Investor, Date, CashFlow

print('Getting net cash flows...')

sheet_MentorTech2005PortfolioReturns = excel_MentorTech2005PortfolioReturns['Net Cash Flows'] #start with getting the net cash flow sheets
sheet_MentorTechIIPortfolioReturns = excel_MentorTechIIPortfolioReturns['Net Cash Flows']
sheet_MentorTechIIIPortfolioReturns = excel_MentorTechIIIPortfolioReturns['Net Cash Flows']
sheet_MentorTechAnnexPortfolioReturns = excel_MentorTechAnnexPortfolioReturns['Net Cash Flows']
sheet_MentorTechJetAnnexPortfolioReturns = excel_MentorTechJetAnnexPortfolioReturns['Net Cash Flows']
sheet_RedandBlueIPortfolioReturns = excel_RedandBlueIPortfolioReturns['Net Cash Flows']
sheet_RedandBlueSSAnnexPortfolioReturns = excel_RedandBlueSSAnnexPortfolioReturns['Net Cash Flows']
sheet_RedandBluePHAnnexPortfolioReturns = excel_RedandBluePHAnnexPortfolioReturns['Net Cash Flows']

sheets = [sheet_MentorTech2005PortfolioReturns, #make a list of our NCF sheets
          sheet_MentorTechIIPortfolioReturns,
          sheet_MentorTechIIIPortfolioReturns,
          sheet_MentorTechAnnexPortfolioReturns,
          sheet_MentorTechJetAnnexPortfolioReturns,
          sheet_RedandBlueIPortfolioReturns,
          sheet_RedandBlueSSAnnexPortfolioReturns,
          sheet_RedandBluePHAnnexPortfolioReturns]

print('Building NetCashFlow data frame...')

def getcashflows(L): #define a function that collects the cashflows from a series of excel sheets

    NetCashFlows = pd.DataFrame(columns=['Investor', 'Date', 'CashFlow']) #create an empty dataframe
    
    for i in L: #for every sheet in our list
    
        for x in range(4, 100000): #for each row "x" starting with 4 and going to some large number like 100,000
            #print(x)
            if i.cell(row = x, column = 1).value == 'zEQUITY VALUE':
                break #if the value of row = i, column = 1 is equal to "zEQUITY VALUE", then break the loop
            NetCashFlows = NetCashFlows.append({'Investor': i.cell(row = x, column = 1).value, #append to data framew
                                  'Date': i.cell(row = x, column = 2).value,
                                  'CashFlow': -float(i.cell(row = x, column = 4).value)}, ignore_index=True)

    return NetCashFlows #spit out the dataframe
        
NetCashFlows = getcashflows(sheets) #call our function on our sheets and save as a variable
    
print(NetCashFlows.head(50))

print("Done")


Getting net cash flows...
Building NetCashFlow data frame...
       Investor       Date    CashFlow
0    Alan Lewis 2005-06-30  -600000.00
1    Alan Lewis 2006-01-23  -450000.00
2    Alan Lewis 2006-09-22  -450000.00
3    Alan Lewis 2006-11-21  -500000.00
4    Alan Lewis 2006-12-28  -500000.00
5    Alan Lewis 2007-04-27  -375000.00
6    Alan Lewis 2007-07-20  -375000.00
7    Alan Lewis 2007-09-13  -500000.00
8    Alan Lewis 2007-11-06  -250000.00
9    Alan Lewis 2008-01-14  -500000.00
10   Alan Lewis 2008-01-17   250000.00
11   Alan Lewis 2008-05-21  1000000.00
12   Alan Lewis 2008-05-22   119323.63
13   Alan Lewis 2008-05-28    73910.00
14   Alan Lewis 2008-06-04  -193233.63
15   Alan Lewis 2008-09-26  -250000.00
16   Alan Lewis 2008-12-19   -55000.00
17   Alan Lewis 2011-04-11  3250000.00
18   Alan Lewis 2011-12-07   305000.00
19   Alan Lewis 2012-11-13   256694.18
20   Alan Lewis 2016-03-17   373821.49
21   Alan Simon 2005-06-30   -40000.00
22   Alan Simon 2006-01-26   -30000.00
23 

In [4]:
#calculate remaining value for each investor, add to NetCash flows dataframe

print("Getting remainder values...")

def getremainders(L):
    
    ResidualValues = pd.DataFrame(columns=['Investor', 'Date', 'CashFlow'])
    
    for x in L:
    
        y = 1 #set a counter variable that will stop to select the column of the spreadsheet where remaining values live
    
        for i in range(1,10000): #find remainder values column
            #print(y)
            if x.cell(row = 3, column = y).value == 'Current Dist-Basis':
                break
            y = y+1

        z = 1 #set a counter variable that will stop to select the column of the spreadsheet with investor accounts

        for j in range(1,10000): #find investor names column
            #print(z)
            if x.cell(row = 3, column = z).value == 'Investor Account':
                break
            z = z+1

        for k in range(4,10000):
            #print(k)
            if x.cell(row = k, column = z).value is None or x.cell(row = k, column = z).value == 'TOTAL':
                break 
            ResidualValues = ResidualValues.append({'Investor': x.cell(row = k, column = z).value, 
                                  'Date': x.cell(row=2, column = y).value,
                                  'CashFlow': float(x.cell(row = k, column = y).value)}, ignore_index=True)  
     
    return ResidualValues


ResidualValues = getremainders(sheets)

print("Done")

frames = [NetCashFlows,ResidualValues]

#concatenate dataframes
TotalCashFlows = pd.concat(frames, sort=False)
TotalCashFlows['Date'] = TotalCashFlows['Date'].dt.date
print(TotalCashFlows.head(100))


print("Now have data frames for Net Cash Flows, Residual Values, and Total Cash Flows")
print("Finished all relevant data type conversions")




Getting remainder values...
Done
              Investor        Date    CashFlow
0           Alan Lewis  2005-06-30  -600000.00
1           Alan Lewis  2006-01-23  -450000.00
2           Alan Lewis  2006-09-22  -450000.00
3           Alan Lewis  2006-11-21  -500000.00
4           Alan Lewis  2006-12-28  -500000.00
5           Alan Lewis  2007-04-27  -375000.00
6           Alan Lewis  2007-07-20  -375000.00
7           Alan Lewis  2007-09-13  -500000.00
8           Alan Lewis  2007-11-06  -250000.00
9           Alan Lewis  2008-01-14  -500000.00
10          Alan Lewis  2008-01-17   250000.00
11          Alan Lewis  2008-05-21  1000000.00
12          Alan Lewis  2008-05-22   119323.63
13          Alan Lewis  2008-05-28    73910.00
14          Alan Lewis  2008-06-04  -193233.63
15          Alan Lewis  2008-09-26  -250000.00
16          Alan Lewis  2008-12-19   -55000.00
17          Alan Lewis  2011-04-11  3250000.00
18          Alan Lewis  2011-12-07   305000.00
19          Alan Lewis  201

In [5]:
#define function xnpv and optimize to get function xirr

def xnpv(rate,cashflows):
    chron_order = sorted(cashflows, key = lambda x: x[0])
    t0 = chron_order[0][0]
    return sum([cf/(1+rate)**((t-t0).days/365.0) for (t,cf) in chron_order])

def xirr(cashflows,guess=0.1):
    return optimize.newton(lambda r: xnpv(r,cashflows),guess, maxiter = 1000000)

In [6]:
#define core output functions

def sumContributions(LP):
    contRows = NetCashFlows.loc[(NetCashFlows['Investor'] == LP) & (NetCashFlows['CashFlow'] < 0)]
    contributions = -sum(contRows['CashFlow'])
    return contributions

def sumDistributions(LP):
    distRows = NetCashFlows.loc[(NetCashFlows['Investor'] == LP) & (NetCashFlows['CashFlow'] > 0)]
    distributions = sum(distRows['CashFlow'])
    return distributions

def sumRemainders(LP):
    remRows = ResidualValues.loc[ResidualValues['Investor'] == LP]
    remainder = sum(remRows['CashFlow'])
    return remainder

def calculateMultiple(LP):
    MoM = (sumDistributions(LP)+sumRemainders(LP))/sumContributions(LP)
    return MoM

def calculateIRR(LP):
    TotalRows = TotalCashFlows.loc[TotalCashFlows['Investor'] == LP]
    
    datesTransactions = list(zip(TotalRows.Date, TotalRows.CashFlow))
    
    IRR = xirr(datesTransactions)
    
    return IRR

#tests:
#print(sumContributions('Brenner, Andrew'))
#print(sumDistributions('Brenner, Andrew'))
#print(sumRemainders('Brenner, Andrew'))
#print(calculateMultiple('Brenner, Andrew'))
#print(calculateIRR('Brenner, Andrew'))


In [7]:
#calculate multiple and IRR for each investor
#(multiple = distributions + remaining value) / contributed

#create a dataframe to hold each investor entities, Contributions, Distributions, Remaining Value, Multiple, IRR 

LPReturns = pd.DataFrame(columns=['Entity', 'Contributions', 'Distributions', 'RV', 'MoM', 'IRR'])

#remove problem cases, let Brett know about these



LPs = TotalCashFlows.Investor.unique() #take an array of all our unique investor entities

#LPs= LPs[(LPs != 'Elaine Woo Camarda')]
#LPs= LPs[(LPs != 'Ason, Jessica')]
#LPs= LPs[(LPs != 'Ason, Jordan')]
#LPs= LPs[(LPs != 'Bortz, Jon')]
#LPs= LPs[(LPs != 'Camarda, Elaine Woo')]
#LPs= LPs[(LPs != 'Ferst, Richard')]
#LPs= LPs[(LPs != 'Carp Family LLC I')]
#LPs= LPs[(LPs != 'Block 3 LLC')]
#LPs= LPs[(LPs != 'JRG2 LLC')]
#LPs= LPs[(LPs != 'Schlidt, Joe')]
#LPs= LPs[(LPs != 'Camarda, Joseph')]
#LPs= LPs[(LPs != 'Elk River/Charles Dilks')] 
#LPs= LPs[(LPs != 'Kalandar, Boris')]
#LPs= LPs[(LPs != 'LFVC')]
#LPs= LPs[(LPs != 'Mentortech Management')]
#LPs= LPs[(LPs != 'Model, Allen')]
#LPs= LPs[(LPs != 'Rowley, Richard')]
#LPs= LPs[(LPs != 'Simon, Fred')]
#LPs= LPs[(LPs != 'Simon, Todd')]
#LPs= LPs[(LPs != 'Urdang, Scott')]

#https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values

for LP in LPs:
    print(LP)
    LPReturns = LPReturns.append({'Entity': LP, 
                        'Contributions': sumContributions(LP),
                        'Distributions': sumDistributions(LP),
                        'RV': sumRemainders(LP),
                        #'MoM': calculateMultiple(LP)
                        #'IRR':calculateIRR(LP)
                                 },ignore_index = True) 

LPReturns['RV'] = LPReturns['RV'].apply(Decimal)
#LPReturns['IRR'] = LPReturns['IRR'].real

print(LPReturns)
#print(LPs)

Alan Lewis
Alan Simon
Allen Model
Bruce Simon
Debbie Gelb
Elaine Woo Camarda
Fred Simon
Gary Erlbaum
George Rohr
Jeff Boily
Joe Camarda
LBCW
Len Lodish
Markowitz A
Markowitz B
Marty Lautman
Michael Aronson
Morris Wilner
MT Management
SPWJ
Stephen Rittenberg
Stephen Simon
Steve Katz
Todd Simon
Albert, Pete
Alper, Robert
Aronson, Elise Trust
Aronson, Michael
Aronson, Rachel Trust
Ason, Jessica
Ason, Jordan
Bernstein, Guy
Bortz, Jon
Camarda, Elaine Woo
Camarda, Joseph
Carp, Brian
Casa II/Weingarten
Cozen, Stephen (IRA)
E & M Sidewater Assoc I LLP
E & M Sidewater Assoc II LLP
Elk River/Charles Dilks
Erlbaum Family
Fairway Investments
Ferst, Richard
Fluhr, Jeffrey
Frankel, Lawrence
GHR
Greenblatt Assoc LP
Greenblatt, Hal
Kalandar, Boris
Kaplin, Ned
Katz, Steve
Klitzberg, Richard
Kosloff, Theodore & Phyliss
Lautman, Martin
Lerner, Andrew
Lerner, David
Lerner, Steven
Lewis, Henry
LFVC
Lodish, Len
Magerman, David
Marks, Joanne Trust
Marks, Stephen
Mason, Terry
Mentortech Management
Model, Alle

In [8]:
LPReturns.to_csv('LP Accounts.csv')